## Week07 Spark application assignment

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,count,desc

# creating spark session

In [0]:
spark = SparkSession.builder.appName("CMS Open Payments Analysis").getOrCreate()


# loading 2023 open payments file

In [0]:
#I uploaded the zip file in dbfs


In [0]:
Zip_path = "/dbfs/FileStore/tables/PGYR2023_P01302025_01212025.zip"

In [0]:
#displaying the files


In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/")) # ensuring that zip file exists


path,name,size,modificationTime
dbfs:/FileStore/tables/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv,OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv,1827342,1742772638000
dbfs:/FileStore/tables/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv,OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv,992593986,1742772786000
dbfs:/FileStore/tables/OP_PGYR2023_README_P01302025.txt,OP_PGYR2023_README_P01302025.txt,5422,1742772638000
dbfs:/FileStore/tables/OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv,OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv,158460,1742772637000
dbfs:/FileStore/tables/PGYR2023_P01302025_01212025.zip,PGYR2023_P01302025_01212025.zip,789005271,1742774422000


In [0]:
# Copying ZIP file from DBFS to local file system for extraction
dbutils.fs.cp("dbfs:/FileStore/tables/PGYR2023_P01302025_01212025.zip", "file:/tmp/openpayments_2023.zip")


Out[22]: True

In [0]:
import zipfile
import os

# Defining  paths
zip_path = "/tmp/openpayments_2023.zip"  # This one is Local ZIP file
extract_path = "/tmp/openpayments_2023"  # This one is Extraction folder

# Ensuring the folder exists
os.makedirs(extract_path, exist_ok=True)

# Extracting  ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Listing  extracted files
os.listdir(extract_path)


Out[23]: ['OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv',
 'OP_PGYR2023_README_P01302025.txt',
 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv']

In [0]:
dbutils.fs.cp("file:/tmp/openpayments_2023", "dbfs:/tmp/openpayments_2023", recurse=True)


Out[24]: True

## reading the CSV file

In [0]:
csv_file_path = "dbfs:/tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv"

df = spark.read.csv(csv_file_path, header=True, inferSchema=True)




+-----------+----------------------+---------------------+--------------------+----------------------+----------------------------+---------------------+----------------------------+-----------------------------+---------------------------+-----------------------------+-----------------------------------------------+-----------------------------------------------+---------------+---------------+------------------+-----------------+------------------+---------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-------------------------------------+-------------------------------------+-------------------------------------+--------------------------------

In [0]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("General_2023_OpenPayments")


# What is the Nature of Payments with reimbursement amounts greater than $1,000 ordered by count?

In [0]:
df.filter(df["Total_Amount_of_Payment_USDollars"] > 1000) \
  .groupBy("Nature_of_Payment_or_Transfer_of_Value") \
  .count() \
  .orderBy("count", ascending=False) \
  .show(10, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count |
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|164092|
|Consulting Fee                                                                                                                                    |105228|
|Travel and Lodging                                                                                                                                |24738 |
|Honoraria                                                      

In [0]:

#Nature of payments with reimbursement amounts greater than 1000 dollars are compensation services, consulting fee, travel and lodging, honoraria, education, royalty or lisence, compensation for serving as faculty, grant, space rental and long term medical supply.

What are the top ten Nature of Payments by count?

In [0]:
df.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
  .count() \
  .orderBy("count", ascending=False) \
  .show(10, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |13378464|
|Travel and Lodging                                                                                                                                |545086  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|236628  |
|Consulting Fee                                     

# Food and Beverage: 13,378,464

Travel and Lodging: 545,086

Compensation for services other than consulting: 236,628

Consulting Fee: 170,630

Education: 161,078

Gift: 31,786

Honoraria: 20,232

Royalty or License: 15,865

Compensation for serving as faculty or a speaker for a medical education program: 12,234

Entertainment: 7,967

# What are the top ten Nature of Payments by total amount?

In [0]:


# Converting  column to numeric type
df = df.withColumn("Total_Amount_of_Payment_USDollars", col("Total_Amount_of_Payment_USDollars").cast("double"))

# Now perform the aggregation
df.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
  .sum("Total_Amount_of_Payment_USDollars") \
  .orderBy("sum(Total_Amount_of_Payment_USDollars)", ascending=False) \
  .show(10, truncate=False)



+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |sum(Total_Amount_of_Payment_USDollars)|
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------+
|Royalty or License                                                                                                                                |1.19217456302E9                       |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|5.946326876500002E8                   |
|Consulting Fee                                             

From the results
Royalty or License: $1.19 billion.

Compensation for services other than consulting: $594.63 million.

Consulting Fee: $514.86 million.

Food and Beverage: $374.49 million.

Travel and Lodging: $179.55 million.

Grant: $111.89 million.

Acquisitions: $71.93 million.

Education: (Details appear incomplete).

# What are the top ten physician specialties by total amount?

In [0]:
df.groupBy("Covered_Recipient_Primary_Type_1") \
  .sum("Total_Amount_of_Payment_USDollars") \
  .orderBy("sum(Total_Amount_of_Payment_USDollars)", ascending=False) \
  .show(10, truncate=False)



+--------------------------------------+--------------------------------------+
|Covered_Recipient_Primary_Type_1      |sum(Total_Amount_of_Payment_USDollars)|
+--------------------------------------+--------------------------------------+
|Medical Doctor                        |2.040183464810028E9                   |
|null                                  |7.933900857900002E8                   |
|Nurse Practitioner                    |1.3546701071000394E8                  |
|Doctor of Osteopathy                  |9.36846884200008E7                    |
|Doctor of Dentistry                   |8.315500003000045E7                   |
|Physician Assistant                   |6.88205734600018E7                    |
|Doctor of Podiatric Medicine          |3.442229346999998E7                   |
|Doctor of Optometry                   |2.3318028930000186E7                  |
|Certified Registered Nurse Anesthetist|2143071.4200000037                    |
|Clinical Nurse Specialist             |

In [0]:
#From the results we can see that the top speciality is medical doctor and 10th position is clinical nurse specialist.

#Who are the top ten physicians by total amount?

In [0]:
df.groupBy("Covered_Recipient_First_Name", "Covered_Recipient_Last_Name") \
  .sum("Total_Amount_of_Payment_USDollars") \
  .orderBy("sum(Total_Amount_of_Payment_USDollars)", ascending=False) \
  .show(10, truncate=False)



+----------------------------+---------------------------+--------------------------------------+
|Covered_Recipient_First_Name|Covered_Recipient_Last_Name|sum(Total_Amount_of_Payment_USDollars)|
+----------------------------+---------------------------+--------------------------------------+
|null                        |null                       |7.933900857900002E8                   |
|STEPHEN                     |BURKHART                   |3.392202493E7                         |
|WILLIAM                     |BINDER                     |2.943437497E7                         |
|KEVIN                       |FOLEY                      |1.73059378E7                          |
|IVAN                        |OSORIO                     |1.606551551E7                         |
|GEORGE                      |MAXWELL                    |1.160032024E7                         |
|ROBERT                      |BOOTH                      |8459167.19                            |
|NEAL               

In [0]:

#The highest total amount is 7.933900857900002E8 which corresponds to the null entry and at 10 th postion is Roger Jackson with 6615711.26